# Computation

If there any error occurs, please visit previous section to check all requirements [←](1_initialization.ipynb). Otherwise please submit an isuue.

## Initial settings

Prior starting the computation all the input conditions are read from settings database (settings.db).

These are database tables and their descriptions:

1. TimePeriod : time span and granularity definition
2. ComputeVariants : definition of variants

As this code runs either way, it defines also functions, so the computation is more understandable

All of below printed can be preferably edited with an sqlite editor, otherwise please use attached python script (test.py)

In [63]:
import pandas as pd
from skyfield.api import load, Topos
from skyfield import timelib
import sqlite3

################################
class Computation:
    def __init__(self, c, o):
        self.centerpoint = c
        self.observed = o
        self.c = c
        self.ts = load.timescale()

    def shift(self, x=0, y=0):
        if pd.isna(x) and pd.isna(y):
            self.centerpoint = self.c
        else:
            self.centerpoint = self.c + Topos(f'{x} N', f'{y} E')

# define functions
def format_time(ts_object):
    if isinstance(ts_object, timelib.Time):
        return ts_object.utc_strftime('%Y-%m-%d %H:%M')
    elif isinstance(ts_object, pd.Timestamp):
        return sel.ts.utc(ts_object.year, ts_object.month, ts_object.day, ts_object.hour, ts_object.minute)
    else:
        return 

def degrees(t):
    # %t%: timestamp value
    return sel.centerpoint.at(format_time(t)).observe(sel.observed).radec()[1].degrees

def distance(t):
    # %t%: timestamp value
    return sel.centerpoint.at(format_time(t)).observe(sel.observed).radec()[-1].au

def frequency(day=0, hour=0, minute=0):
    if minute:
        return '1min'
    elif hour:
        return '60min'
    else:
        return '1440min'  # 1 day timestep
    
    
def observer(start, end, gran):
    """iteration that computes values for given parameters
    %c%: center point position (with Topos module or another) 
    %start%: start date (pandas dataframe row)
    %end%: end date (pandas dataframe row)
    %gran%: granularity (pandas dataframe row)
    returns pandas dataframe
    """
    if not isinstance(start, pd.DataFrame) or not isinstance(end, pd.DataFrame) or not isinstance(gran, pd.DataFrame):
        return
    print(f"""      from {start.Day.iloc[0]}/{start.Month.iloc[0]}/{start.Year.iloc[0]} 
        to {end.Day.iloc[0]}/{end.Month.iloc[0]}/{end.Year.iloc[0]} 
        at time step {gran.Day.iloc[0]} days {gran.Hour.iloc[0]} hours {gran.Minute.iloc[0]} minutes""")
    time_span = pd.date_range(f'{start.Month.iloc[0]}/{start.Day.iloc[0]}/{start.Year.iloc[0]}',
                             f'{end.Month.iloc[0]}/{end.Day.iloc[0]}/{end.Year.iloc[0]}',
                             freq=frequency(gran.Day.iloc[0], gran.Hour.iloc[0], gran.Minute.iloc[0]))
    
    obs = pd.DataFrame(time_span, columns=['date_time'])
    obs['declination'] = obs['date_time'].apply(degrees)
    obs['distance'] = obs['date_time'].apply(distance)
    
    return obs
################################

db_connection = sqlite3.connect("./settings.db")
time_period = pd.read_sql("select * from TimePeriod", db_connection)
variants = pd.read_sql("select * from ComputeVariants", db_connection)
labels = variants[variants.columns[[2,3,4]]].apply(lambda x: f'{list(x)[0]} ({list(x)[1]} / {list(x)[2]})', 1).tolist()

start_point = time_period['Param'] == 'start_point'
end_point = time_period['Param'] == 'end_point'
granularity = time_period['Param'] == 'granularity'

print("Time settings verificaion:")
print(f"  Start: {time_period['Day'][start_point].values[0]}. {time_period['Month'][start_point].values[0]}. {time_period['Year'][start_point].values[0]}")
print(f"  Finish: {time_period['Day'][end_point].values[0]}. {time_period['Month'][end_point].values[0]}. {time_period['Year'][end_point].values[0]}")
print(f"  Time step: {time_period['Day'][granularity].values[0]} days {time_period['Hour'][granularity].values[0]} hours {time_period['Minute'][granularity].values[0]} minutes")

Time settings verificaion:
  Start: 1. 1. 2022
  Finish: 1. 2. 2022
  Time step: 0 days 1 hours 0 minutes


## Let's run



...

In [64]:
planets = load('./source/de430.bsp')
sel = Computation(planets['earth'], planets['moon'])

for idx, var in variants.iterrows():
    if not pd.isna(var[3]):
            sel.shift(var[3], var[4])
    print(f'computing variant {var[1]} {labels[idx]}')
    o = observer(time_period[start_point], time_period[end_point], time_period[granularity])
    print(f'      output to file ./result/{var[-1]}')
    o.to_csv(f'./result/{var[-1]}', index=False, mode='w')


computing variant Earth Barycenter (nan / nan)
      from 1/1/2022 
        to 1/2/2022 
        at time step 0 days 1 hours 0 minutes
      output to file ./result/computed.csv
computing variant Earth NewYork (40.0 / -75.0)
      from 1/1/2022 
        to 1/2/2022 
        at time step 0 days 1 hours 0 minutes
      output to file ./result/computed_ny.csv
computing variant Earth Paris (50.0 / 0.0)
      from 1/1/2022 
        to 1/2/2022 
        at time step 0 days 1 hours 0 minutes
      output to file ./result/computed_pr.csv
computing variant Earth Prague (50.0 / 15.0)
      from 1/1/2022 
        to 1/2/2022 
        at time step 0 days 1 hours 0 minutes
      output to file ./result/computed_pg.csv
computing variant Earth RioDeJaneiro (-25.0 / -45.0)
      from 1/1/2022 
        to 1/2/2022 
        at time step 0 days 1 hours 0 minutes
      output to file ./result/computed_rj.csv
computing variant Earth CapeTown (-35.0 / 20.0)
      from 1/1/2022 
        to 1/2/2022 
        

## Further analysis

You can now continue to analysis part [→](3_analysis.ipynb).
